# 「動物園の飼育員」ボット
StreamlitとChatGPT APIを使い、動物についての情報を教えるチャットボットを構築します。  

## ライブラリのインストール
Streamlit、およびアプリの動作の確認に使用する「ngrok」をインストールします。  
また、ChatGPT APIを使用するために必要なライブラリ、openaiをインストールします。  

In [2]:
!pip install streamlit==1.20.0 --quiet
!pip install pyngrok==4.1.1 --quiet
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.4 MB/s eta 0:00:00


インストールの完了後、streamlit、ngrok、およびopenaiをインポートしておきます。

In [3]:
import streamlit as st
from pyngrok import ngrok
import openai

## 画像のアップロード
チャットボットに使用するイメージ画像をアップロードします。  
教材をダウンロードし、「Zoo.png」を画面左の「ファイル」にドラッグアンドドロップしましょう。  

## チャットボットのコード
`%%writefile`のマジックコマンドを使って、チャットボットのコードを「app.py」に書き込みます。  
`sysytem`の`content`にチャットボットの機能や性格などの設定を記述しますが、意図しない動作を避けるために「やらないこと」（ネガティブプロンプト）も記述しましょう。

In [4]:
%%writefile app.py
# 以下を「app.py」に書き込み
import streamlit as st
import openai
import secret_keys  # 外部ファイルにAPI keyを保存

openai.api_key = secret_keys.openai_api_key

system_prompt = """
あなたは優秀な動物園の飼育員です。
動物の生態や、動物の種類、動物の特性、動物の生息地など動物園に来た人たちに対して教えてあげてください。
あなたは動物園の飼育員なので、例えば以下のような動物に関すること以外のことを聞かれても、絶対に答えないでください。

* 戦争
* 芸能人
* 科学
* 歴史
* プログラミング
* 政治
"""

# st.session_stateを使いメッセージのやりとりを保存
if "messages" not in st.session_state:
    st.session_state["messages"] = [
        {"role": "system", "content": system_prompt}
        ]

# チャットボットとやりとりする関数
def communicate():
    messages = st.session_state["messages"]

    user_message = {"role": "user", "content": st.session_state["user_input"]}
    messages.append(user_message)

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    bot_message = response["choices"][0]["message"]
    messages.append(bot_message)

    st.session_state["user_input"] = ""  # 入力欄を消去


# ユーザーインターフェイスの構築
st.title("私は優秀な動物園の飼育員です。")
st.image("Zoo.png")
st.write("動物についてなんでも聞いてください!")

user_input = st.text_input("メッセージを入力してください。", key="user_input", on_change=communicate)

if st.session_state["messages"]:
    messages = st.session_state["messages"]

    for message in reversed(messages[1:]):  # 直近のメッセージを上に
        speaker = "🙂"
        if message["role"]=="assistant":
            speaker="🤖"

        st.write(speaker + ": " + message["content"])

Writing app.py


## OpenAIのAPI keyを設定
ChatGPT APIを使用するために必要な「API key」を設定します。  
`%%writefile`のマジックコマンドを使って、API keyを設定するコードを「secret_keys.py」に書き込みます。  
以下のコードの、  
`openai_api_key = "Your API key"`  
における  
`Your API key`の箇所を、自分のAPI keyに置き換えます。  
ChatGPTのAPI keyは、OpenAIのサイトで取得できます。   
https://platform.openai.com/account/api-keys


In [5]:
%%writefile secret_keys.py

openai_api_key = "Your API key"

Writing secret_keys.py


API keyの流出にはリスクがあります。  
他者に知られないように、慎重に扱ってください。

## ngrokのAuthtokenを設定
ngrokで接続するために必要な「Authtoken」を設定します。  
以下のコードの、  
`!ngrok authtoken YourAuthtoken`  
における  
`YourAuthtoken`の箇所を、自分のAuthtokenに置き換えます。  
Authtokenは、ngrokのサイトに登録すれば取得することができます。  
https://ngrok.com/


In [6]:
!ngrok authtoken YourAuthtoken

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


ngrokのAuthtokenも、他者に知られないように慎重に扱ってください。

## アプリの起動
streamlitの`run`コマンドでアプリを起動します。


In [7]:
!streamlit run app.py &>/dev/null&  # 「&>/dev/null&」により、出力を非表示にしてバックグランドジョブとして実行

ngrokのプロセスを終了した上で、新たにポートを指定して接続します。  
接続の結果、urlを取得できます。  
ngrokの無料プランでは同時に1つのプロセスしか動かせないので、エラーが発生した場合は「ランタイム」→「セッションの管理」で不要なGoogle Colabのセッションを修了しましょう。  

In [8]:
ngrok.kill()  # プロセスの修了
url = ngrok.connect(port="8501")  # 接続

INFO:pyngrok.process:ngrok process starting: 1364
2023-07-14 02:16:52.339 INFO    pyngrok.process: ngrok process starting: 1364
INFO:pyngrok.process:t=2023-07-14T02:16:52+0000 lvl=info msg="no configuration paths supplied"

2023-07-14 02:16:52.361 INFO    pyngrok.process: t=2023-07-14T02:16:52+0000 lvl=info msg="no configuration paths supplied"

INFO:pyngrok.process:t=2023-07-14T02:16:52+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml

2023-07-14 02:16:52.369 INFO    pyngrok.process: t=2023-07-14T02:16:52+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml

INFO:pyngrok.process:t=2023-07-14T02:16:52+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil

2023-07-14 02:16:52.376 INFO    pyngrok.process: t=2023-07-14T02:16:52+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil

INFO:pyngrok.process:t=2023-07-14T02:16:52+0000 lvl=info msg="starting web service" obj=

## 動作の確認

URLのhttpの部分をhttpsに変換する関数を設定します。

In [9]:
def convert_http_to_https(url):
    if url.startswith("http://"):
        url = url.replace("http://", "https://", 1)
    return url

2023-07-14 02:16:52.843 INFO    pyngrok.process: t=2023-07-14T02:16:52+0000 lvl=info msg=end pg=/api/tunnels id=24bbf14cb462641f status=201 dur=157.362509ms



変換したurlを表示し、リンク先でチャットボットが動作することを確認します。

In [10]:
print(convert_http_to_https(url))

https://af46-34-125-111-40.ngrok-free.app


チャットボットの動作確認後、OpenAIのサイトでAPIの使用量を確認してみましょう。  
https://platform.openai.com/account/usage